In [1]:
import pandas as pd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

C:\Users\Usuario\AppData\Local\Temp\ipykernel_22596\2564754530.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(ruta + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(ruta + '/_keys/llave_db_agrocittca', password='micontrasena')
# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']
# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [5]:
# funcon para ejecutar una script SQL
def dbQuery(q):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [6]:
usuarios_act = dbQuery('select tok.created_at, usu.codigo, usu.name, usu.apellido_paterno, usu.apellido_materno, username from oauth_access_tokens tok\
                        inner join users usu on tok.user_id=usu.id')

In [8]:
lista_users_act = pd.DataFrame(usuarios_act)

In [9]:
lista_users_act

,created_at,codigo,name,apellido_paterno,apellido_materno,username
0,2023-02-10 13:57:02,#4,#4 - usuario01 ap,ap,ap,usuario01
1,2023-09-26 14:14:57,12478,12478 - HUGO SERRATE,SERRATE,COLAMARINO,hserrate
2,2023-12-12 20:37:29,42180,42180 - JOSE EDGAR PRADO,PRADO,HUANCA,42180
3,2023-01-26 10:06:52,#15,#15 - UTEA AP,AP,AP,utea
4,2023-02-16 09:54:00,#15,#15 - UTEA AP,AP,AP,utea
...,...,...,...,...,...,...
1008,2023-12-20 21:18:12,5410,5410 - SANTOS HUALLPA,HUALLPA,MAMANI,5410
1009,2023-02-25 13:23:07,#15,#15 - UTEA AP,AP,AP,utea
1010,2023-12-20 19:24:13,8814,8814 - WALTER DANIEL MURIALDO,MURIALDO,MENDEZ,8814
1011,2023-03-31 08:11:35,13203,13203 - MARCOS EDUARDO SGARABOTTO,SGARABOTTO,MENDEZ,13203
